In [4]:
import requests
respuesta = requests.get('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')


In [5]:
respuesta.text

'long,lat,id,nombre,categoria,cocina,ambientacion,telefono,mail,horario,calle_nombre,calle_altura,calle_cruce,direccion_completa,barrio,comuna,codigo_postal,codigo_postal_argentino\r\n-58.3633067544777,-34.6229634698018,1602,GUANTANAMERA,RESTAURANTE,CUBANA,MODERNO,4314 0770,,,MOREAU DE JUSTO ALICIA,2092,,"MOREAU DE JUSTO, ALICIA AV. 2092",Puerto Madero,Comuna 1,1107,C1107AFP\r\n-58.3923296263654,-34.6215535313181,676,1843 VINACOTECA,VINERIA,,,4779 0806,,lunes a domingo de 8 a 00,HUMBERTO 1Âº,1843,,HUMBERTO 1Âº 1843,San Cristobal,Comuna 3,,\r\n-58.4535422659946,-34.5603113496909,487,2080 EXPRESS,RESTAURANTE,TRADICIONAL,CLASICO PORTEÃ\x91O,4788 4500,,lunes a domingo de 8 a 00,JURAMENTO,2080,,JURAMENTO AV. 2080,Belgrano,Comuna 13,1428,C1428DNH\r\n-58.4754910533597,-34.6770507014194,2737,22,CAFE,MINUTAS,MODERNO,,,,MURGUIONDO,4188,,MURGUIONDO 4188,Villa Lugano,Comuna 8,1439,C1439FTB\r\n-58.4254372025985,-34.5856950120038,2,4 EN 2 I,RESTAURANTE,,,4832 9510,,,BORGES JORGE LUIS,2095,,"BORGES, 

In [6]:
import pymysql
conexion = pymysql.connect(
    host='localhost',
    user='root',
    passwd='root',
    database='ofertas_gastronomicas') 
cursor = conexion.cursor()

In [7]:
cursor.execute ("""
               CREATE TABLE local (
               id_local INT NOT NULL auto_increment,
               nombre varchar(80) NOT NULL,
               categoria varchar(80) NOT NULL,
               direccion varchar(80) NOT NULL,
               barrio varchar(80) NOT NULL,
               comuna varchar(80) NOT NULL,
               primary key (id_local));
               """)

0

In [8]:
conexion.commit()

In [9]:
import csv
from io import StringIO
doc = csv.reader(StringIO(respuesta.text))

In [10]:
print(next(doc))

['long', 'lat', 'id', 'nombre', 'categoria', 'cocina', 'ambientacion', 'telefono', 'mail', 'horario', 'calle_nombre', 'calle_altura', 'calle_cruce', 'direccion_completa', 'barrio', 'comuna', 'codigo_postal', 'codigo_postal_argentino']


In [11]:
lista = []
for fila in doc:
    lista.append((fila[2],fila[3],fila[4],fila[13],fila[14],fila[15]))

In [12]:
lista

[('1602',
  'GUANTANAMERA',
  'RESTAURANTE',
  'MOREAU DE JUSTO, ALICIA AV. 2092',
  'Puerto Madero',
  'Comuna 1'),
 ('676',
  '1843 VINACOTECA',
  'VINERIA',
  'HUMBERTO 1Âº 1843',
  'San Cristobal',
  'Comuna 3'),
 ('487',
  '2080 EXPRESS',
  'RESTAURANTE',
  'JURAMENTO AV. 2080',
  'Belgrano',
  'Comuna 13'),
 ('2737', '22', 'CAFE', 'MURGUIONDO 4188', 'Villa Lugano', 'Comuna 8'),
 ('2',
  '4 EN 2 I',
  'RESTAURANTE',
  'BORGES, JORGE LUIS 2095',
  'Palermo',
  'Comuna 14'),
 ('3', '4 X 4 CAFE', 'BAR', 'DEL LIBERTADOR AV. 3838', 'Palermo', 'Comuna 14'),
 ('2091',
  '5TA AVENIDA',
  'RESTAURANTE',
  'SARMIENTO 814',
  'San Nicolas',
  'Comuna 1'),
 ('2093',
  '5TA AVENIDA I',
  'RESTAURANTE',
  'SARMIENTO 381',
  'San Nicolas',
  'Comuna 1'),
 ('2655', '5TA ZONA', '', 'MURILLO 1049', 'Villa Crespo', 'Comuna 15'),
 ('2092', '650', 'RESTAURANTE', 'SAN MARTIN 650', 'San Nicolas', 'Comuna 1'),
 ('1636',
  '788 FOODBAR',
  'RESTAURANTE',
  'ARENALES 1877',
  'Recoleta',
  'Comuna 2'),
 ('

In [13]:
cursor.executemany('INSERT INTO local VALUES(%s,%s,%s,%s,%s,%s)',lista)

2823

In [16]:
conexion.commit()

In [33]:
cursor.execute('SELECT barrio,count(*) PUBS FROM local WHERE categoria = "BAR" GROUP BY barrio  ORDER BY PUBS DESC LIMIT 1')
pubsporbarrios = cursor.fetchall()
pubsporbarrios

(('San Nicolas', 63),)

In [28]:
cursor.execute('SELECT categoria,count(*) locales FROM local GROUP BY categoria')
localesporcategoria = cursor.fetchall()
localesporcategoria

(('RESTAURANTE', 1990),
 ('BAR', 247),
 ('VINERIA', 22),
 ('CAFE', 334),
 ('PUB', 53),
 ('', 116),
 ('CONFITERIA', 55),
 ('SANDWICHERIA', 4),
 ('DELIVERY & TAKE AWAY', 2))

In [36]:
cursor.execute('SELECT categoria,comuna,count(*) restaurante FROM  local WHERE categoria = "RESTAURANTE" GROUP BY comuna  ')
restauranteporporcomuna = cursor.fetchall()
restauranteporporcomuna

(('RESTAURANTE', 'Comuna 14', 366),
 ('RESTAURANTE', 'Comuna 15', 57),
 ('RESTAURANTE', 'Comuna 2', 227),
 ('RESTAURANTE', 'Comuna 1', 609),
 ('RESTAURANTE', 'Comuna 13', 108),
 ('RESTAURANTE', 'Comuna 3', 145),
 ('RESTAURANTE', 'Comuna 12', 42),
 ('RESTAURANTE', 'Comuna 11', 33),
 ('RESTAURANTE', 'Comuna 5', 56),
 ('RESTAURANTE', 'Comuna 6', 87),
 ('RESTAURANTE', 'Comuna 4', 95),
 ('RESTAURANTE', 'Comuna 7', 64),
 ('RESTAURANTE', 'Comuna 10', 43),
 ('RESTAURANTE', 'Comuna 9', 52),
 ('RESTAURANTE', 'Comuna 8', 6))